# Import all the stuff

In [6]:
!pip install bert-serving-client
!pip install pandas
!pip install tensorflow==1.15.0
!pip install matplotlib
!pip install keras

import random
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializers import RandomUniform

from bert_serving.client import BertClient


ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.0
ERROR: No matching distribution found for tensorflow==1.15.0


In [7]:
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[ 0.13186474,  0.32404128, -0.82704437, ..., -0.3711958 ,
        -0.39250174, -0.31721866],
       [ 0.24873531, -0.12334424, -0.38933852, ..., -0.44756213,
        -0.5591355 , -0.11345179],
       [ 0.28627345, -0.18580122, -0.30906814, ..., -0.2959366 ,
        -0.39310536,  0.07640187]], dtype=float32)

# Testing the code

In [8]:
x_input = np.array([[1,2,3,4,5]])
y_input = np.array([[10]])

model = Sequential()
model.add(Dense(units=32, activation="tanh", input_dim=x_input.shape[1], kernel_initializer='random_normal'))
model.add(Dense(units=1, kernel_initializer='random_normal'))

model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                192       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


# Run a model

In [ ]:
print("allo")